<a href="https://colab.research.google.com/github/Sahilvaghasiyaa/-Market-Trend-classifier-and-analyzer/blob/main/CV%3E%3E%3E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install yfinance pandas numpy mplfinance Pillow ta matplotlib

# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
from PIL import Image
import os
from ta.momentum import StochRSIIndicator, RSIIndicator
from ta.trend import EMAIndicator, SMAIndicator
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import shutil
from google.colab import files

# 📂 Specify the folder you want to save
folder_to_save = "sellclass"  # Change this to your folder name
zip_filename = f"{folder_to_save}.zip"

# 📦 Zip the folder
shutil.make_archive(folder_to_save, 'zip', folder_to_save)

# 📥 Download the zipped folder
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:


class TradingDataCollector:
    def __init__(self):
        # Initialize instruments first
        self.instruments = {
                  # US Large Caps & Tech
                  'AAPL': 'Apple', 'MSFT': 'Microsoft', 'GOOGL': 'Google',
                  'AMZN': 'Amazon', 'NVDA': 'NVIDIA', 'META': 'Meta',
                  'TSLA': 'Tesla', 'AMD': 'AMD', 'JPM': 'JPMorgan',
                  'V': 'Visa', 'MA': 'Mastercard', 'PYPL': 'PayPal',
                  'NFLX': 'Netflix', 'DIS': 'Disney', 'ADBE': 'Adobe',
                  'ORCL': 'Oracle', 'CRM': 'Salesforce', 'INTC': 'Intel',
                  'BAC': 'Bank of America', 'WMT': 'Walmart',
                  'PG': 'P&G', 'KO': 'Coca-Cola', 'PEP': 'PepsiCo',
                  'MCD': 'McDonalds', 'NKE': 'Nike',

                  # Indian Large Caps (High Volume)
                  'RELIANCE.NS': 'Reliance Industries', 'TCS.NS': 'TCS', 'INFY.NS': 'Infosys',
                  'HDFCBANK.NS': 'HDFC Bank', 'ICICIBANK.NS': 'ICICI Bank',
                  'SBIN.NS': 'State Bank of India', 'BHARTIARTL.NS': 'Bharti Airtel',
                  'KOTAKBANK.NS': 'Kotak Mahindra Bank', 'LT.NS': 'Larsen & Toubro',
                  'ITC.NS': 'ITC', 'HINDUNILVR.NS': 'Hindustan Unilever', 'ASIANPAINT.NS': 'Asian Paints',
                  'MARUTI.NS': 'Maruti Suzuki', 'BAJFINANCE.NS': 'Bajaj Finance',
                  'ADANIENT.NS': 'Adani Enterprises', 'TITAN.NS': 'Titan', 'ULTRACEMCO.NS': 'UltraTech Cement',
                  'WIPRO.NS': 'Wipro', 'M&M.NS': 'Mahindra & Mahindra',

                  # Major Stock Indices (High Volume)
                  '^GSPC': 'S&P 500', '^DJI': 'Dow Jones Industrial Average', '^IXIC': 'Nasdaq Composite',
                  '^RUT': 'Russell 2000', '^VIX': 'CBOE Volatility Index', '^FTSE': 'FTSE 100',
                  '^GDAXI': 'DAX 40', '^FCHI': 'CAC 40', '^N225': 'Nikkei 225',
                  '^HSI': 'Hang Seng Index', '^AXJO': 'ASX 200', '^STI': 'Straits Times Index',
                  '^NSEI': 'Nifty 50', '^BSESN': 'Sensex', '^NSEBANK': 'Nifty Bank',
                  '^CNXIT': 'Nifty IT', '^CNXMIDCAP': 'Nifty Midcap 100',

                  # High-Volume Commodities & Metals
                  'GC=F': 'Gold Futures', 'SI=F': 'Silver Futures', 'HG=F': 'Copper Futures',
                  'PL=F': 'Platinum Futures', 'PA=F': 'Palladium Futures', 'CL=F': 'Crude Oil Futures',
                  'NG=F': 'Natural Gas Futures', 'ZC=F': 'Corn Futures', 'ZW=F': 'Wheat Futures',
                  'ZS=F': 'Soybean Futures', 'ZL=F': 'Soybean Oil Futures', 'KC=F': 'Coffee Futures',
                  'CT=F': 'Cotton Futures',

                  # Forex Major Pairs (High Liquidity)
                  'EURUSD=X': 'EUR/USD', 'GBPUSD=X': 'GBP/USD', 'USDJPY=X': 'USD/JPY',
                  'AUDUSD=X': 'AUD/USD', 'USDCAD=X': 'USD/CAD', 'NZDUSD=X': 'NZD/USD',
                  'USDCHF=X': 'USD/CHF', 'EURJPY=X': 'EUR/JPY', 'GBPJPY=X': 'GBP/JPY',
                  'EURGBP=X': 'EUR/GBP', 'USDHKD=X': 'USD/HKD', 'USDSGD=X': 'USD/SGD',
                  'USDZAR=X': 'USD/ZAR', 'USDNOK=X': 'USD/NOK', 'EURCAD=X': 'EUR/CAD',
                  'AUDJPY=X': 'AUD/JPY', 'USDSEK=X': 'USD/SEK', 'EURCHF=X': 'EUR/CHF',

                  # Crypto (High Volume)
                  'BTC-USD': 'Bitcoin', 'ETH-USD': 'Ethereum', 'BNB-USD': 'Binance',
                  'SOL-USD': 'Solana', 'XRP-USD': 'Ripple', 'ADA-USD': 'Cardano',
                  'DOGE-USD': 'Dogecoin', 'MATIC-USD': 'Polygon', 'LINK-USD': 'Chainlink',
                  'DOT-USD': 'Polkadot', 'AVAX-USD': 'Avalanche', 'SHIB-USD': 'Shiba Inu',
                  'UNI-USD': 'Uniswap', 'ATOM-USD': 'Cosmos', 'TRX-USD': 'Tron',
                  'LTC-USD': 'Litecoin', 'BCH-USD': 'Bitcoin Cash', 'XLM-USD': 'Stellar',
                  'ICP-USD': 'Internet Computer', 'APT-USD': 'Aptos', 'ARB-USD': 'Arbitrum',
                  'OP-USD': 'Optimism'

        }

        # Adjusted timeframes to comply with Yahoo Finance limits
        self.timeframes = {
            '5m': '60d',    # Maximum allowed for 5m
            '15m': '60d',   # Maximum allowed for 15m
            '30m': '60d',   # Maximum allowed for 30m
            '1h': '730d'    # Maximum allowed for 1h (2 years)
        }

        # Main directories
        self.base_dir = 'buyclass'
        self.dirs = {
            'input': f'{self.base_dir}/buyinput',
            'output': f'{self.base_dir}/buyoutput'
        }

        # Create directories
        for path in self.dirs.values():
            os.makedirs(path, exist_ok=True)

        # Sequential image counter
        self.image_counter = 1

        # Date tracking
        self.collected_dates = {
            f"{symbol}_{timeframe}": set()
            for symbol in self.instruments
            for timeframe in self.timeframes
        }

    def calculate_indicators(self, df):
        """Calculate technical indicators for analysis"""
        try:
            # Make sure we have OHLCV data
            required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
            if not all(col in df.columns for col in required_columns):
                print("Missing required price data columns")
                return None

            # Calculate EMAs
            df['EMA9'] = df['Close'].ewm(span=9, adjust=False).mean()
            df['EMA21'] = df['Close'].ewm(span=21, adjust=False).mean()
            df['EMA55'] = df['Close'].ewm(span=55, adjust=False).mean()
            df['EMA89'] = df['Close'].ewm(span=89, adjust=False).mean()

            # Calculate Stochastic
            period = 14
            df['LowestLow'] = df['Low'].rolling(window=period).min()
            df['HighestHigh'] = df['High'].rolling(window=period).max()
            df['%K'] = ((df['Close'] - df['LowestLow']) /
                       (df['HighestHigh'] - df['LowestLow'])) * 100
            df['%D'] = df['%K'].rolling(window=3).mean()

            # Calculate ATR for volatility
            df['TR'] = np.maximum(
                df['High'] - df['Low'],
                np.maximum(
                    abs(df['High'] - df['Close'].shift()),
                    abs(df['Low'] - df['Close'].shift())
                )
            )
            df['ATR'] = df['TR'].rolling(window=14).mean()

            # Calculate Fibonacci levels
            df['High_Roll'] = df['High'].rolling(window=55).max()
            df['Low_Roll'] = df['Low'].rolling(window=55).min()
            range_size = df['High_Roll'] - df['Low_Roll']

            # Fibonacci retracement levels
            df['Fib_618'] = df['High_Roll'] - (range_size * 0.618)
            df['Fib_500'] = df['High_Roll'] - (range_size * 0.500)
            df['Fib_382'] = df['High_Roll'] - (range_size * 0.382)

            return df

        except Exception as e:
            print(f"Error calculating indicators: {str(e)}")
            return None

    def check_ema_condition(self, df, index):
        """Check if EMAs are properly aligned"""
        try:
            current = df.iloc[index]
            return (current['EMA9'] > current['EMA21'] >
                   current['EMA55'] > current['EMA89'])
        except Exception as e:
            print(f"Error checking EMA condition: {str(e)}")
            return False

    def check_golden_zone(self, df, index):
        """Check if price is in the golden zone (0.382-0.618)"""
        try:
            current = df.iloc[index]
            price = current['Close']
            return (current['Fib_618'] <= price <= current['Fib_382'])
        except Exception as e:
            print(f"Error checking golden zone: {str(e)}")
            return False

    def check_discount_zone(self, df, index):
        """Check if price is in the discount zone (0.618-0.786)"""
        try:
            current = df.iloc[index]
            price = current['Close']
            fib_786 = current['High_Roll'] - ((current['High_Roll'] -
                      current['Low_Roll']) * 0.786)
            return (fib_786 <= price <= current['Fib_618'])
        except Exception as e:
            print(f"Error checking discount zone: {str(e)}")
            return False

    def check_stochastic(self, df, index):
        """Check if stochastic indicates oversold conditions"""
        try:
            current = df.iloc[index]
            return (20 <= current['%K'] <= 40 and
                   20 <= current['%D'] <= 40)
        except Exception as e:
            print(f"Error checking stochastic: {str(e)}")
            return False

    def check_ema_crossover(self, df, index):
        """Check for bullish EMA crossover"""
        try:
            if index < 1:
                return False

            current = df.iloc[index]
            previous = df.iloc[index-1]

            return (previous['EMA9'] <= previous['EMA21'] and
                   current['EMA9'] > current['EMA21'])
        except Exception as e:
            print(f"Error checking EMA crossover: {str(e)}")
            return False

    def is_date_collected(self, symbol, timeframe, timestamp):
        """Check if we already collected data for this date"""
        key = f"{symbol}_{timeframe}"
        date_str = timestamp.strftime('%Y-%m-%d')

        if date_str in self.collected_dates[key]:
            return True

        self.collected_dates[key].add(date_str)
        return False

    def check_all_conditions(self, df, index):
        """Check all possible combinations of conditions"""
        try:
            # Base condition (Required)
            ema_ok = self.check_ema_condition(df, index)
            if not ema_ok:
                return False

            # Check all other conditions
            golden_zone = self.check_golden_zone(df, index)
            discount_zone = self.check_discount_zone(df, index)
            stoch_ok = self.check_stochastic(df, index)
            crossover = self.check_ema_crossover(df, index)

            # All possible combinations with base condition
            combinations = [
                (golden_zone,),                                    # 1+2
                (discount_zone,),                                 # 1+3
                (stoch_ok,),                                     # 1+4
                (crossover,),                                    # 1+5
                (golden_zone, discount_zone),                    # 1+2+3
                (golden_zone, stoch_ok),                        # 1+2+4
                (golden_zone, crossover),                       # 1+2+5
                (discount_zone, stoch_ok),                      # 1+3+4
                (discount_zone, crossover),                     # 1+3+5
                (stoch_ok, crossover),                         # 1+4+5
                (golden_zone, discount_zone, stoch_ok),        # 1+2+3+4
                (golden_zone, discount_zone, crossover),       # 1+2+3+5
                (golden_zone, stoch_ok, crossover),           # 1+2+4+5
                (discount_zone, stoch_ok, crossover),         # 1+3+4+5
                (golden_zone, discount_zone, stoch_ok, crossover) # 1+2+3+4+5
            ]

            # Return True if any combination is satisfied
            return any(all(cond for cond in combo) for combo in combinations)

        except Exception as e:
            print(f"Error checking conditions: {str(e)}")
            return False

    def save_chart_images(self, df, index):
        """Save input and output images"""
        try:
            # Input image (200 candles)
            start_idx = max(0, index - 199)
            input_data = df.iloc[start_idx:index+1]

            if len(input_data) != 200:
                return False

            # Output image (200 candles: 100 deleted + 100 future)
            if index + 100 >= len(df):
                return False

            output_data = df.iloc[index-99:index+101]
            if len(output_data) != 200:
                return False

            # Chart style
            mc = mpf.make_marketcolors(
                up='green',
                down='red',
                edge='inherit',
                wick='inherit'
            )
            s = mpf.make_mpf_style(
                marketcolors=mc,
                gridstyle='',
                y_on_right=True
            )

            # Save input image
            fig_input, _ = mpf.plot(
                input_data,
                type='candle',
                style=s,
                volume=False,
                returnfig=True,
                figsize=(10, 10),
                tight_layout=True,
                show_nontrading=False
            )

            input_path = os.path.join(self.dirs['input'], f'inputimg{self.image_counter}.png')
            fig_input.savefig(input_path, dpi=51.2, bbox_inches='tight', pad_inches=0.1)
            plt.close(fig_input)

            # Save output image
            fig_output, _ = mpf.plot(
                output_data,
                type='candle',
                style=s,
                volume=False,
                returnfig=True,
                figsize=(10, 10),
                tight_layout=True,
                show_nontrading=False
            )

            output_path = os.path.join(self.dirs['output'], f'outputimg{self.image_counter}.png')
            fig_output.savefig(output_path, dpi=51.2, bbox_inches='tight', pad_inches=0.1)
            plt.close(fig_output)

            # Resize both images
            for path in [input_path, output_path]:
                img = Image.open(path)
                img = img.resize((512, 512), Image.Resampling.LANCZOS)
                img.save(path)

            self.image_counter += 1
            return True
        except Exception as e:
            print(f"Error saving charts: {str(e)}")
            return False

    def collect_data(self):
        """Main data collection process"""
        print("Starting data collection process...")
        start_time = time.time()

        for symbol in self.instruments:
            if self.image_counter > 5000:
                break

            print(f"\nProcessing {self.instruments[symbol]}")

            for timeframe, period in self.timeframes.items():
                try:
                    print(f"Fetching {timeframe} data...")
                    ticker = yf.Ticker(symbol)
                    df = ticker.history(interval=timeframe, period=period)

                    if len(df) < 300:
                        print(f"Not enough data: {len(df)} candles")
                        continue

                    print("Calculating indicators...")
                    df = self.calculate_indicators(df)
                    if df is None:
                        continue

                    print("Processing candles...")
                    for i in range(200, len(df)-100):
                        if self.image_counter > 5000:
                            break

                        # Check date duplication
                        timestamp = pd.Timestamp(df.index[i])
                        if self.is_date_collected(symbol, timeframe, timestamp):
                            continue

                        # Check all condition combinations
                        if self.check_all_conditions(df, i):
                            # Save synchronized images
                            if self.save_chart_images(df, i):
                                print(f"Saved image pair {self.image_counter}/5000")

                except Exception as e:
                    print(f"Error processing {symbol} {timeframe}: {str(e)}")
                    continue

        end_time = time.time()
        print(f"\nCollection Complete!")
        print(f"Total image pairs collected: {self.image_counter-1}")
        print(f"Time taken: {(end_time - start_time) / 60:.2f} minutes")

if __name__ == "__main__":
    collector = TradingDataCollector()
    collector.collect_data()



Starting data collection process...

Processing Apple
Fetching 5m data...
Calculating indicators...
Processing candles...
Saved image pair 2/5000
Saved image pair 3/5000
Saved image pair 4/5000
Saved image pair 5/5000
Saved image pair 6/5000
Saved image pair 7/5000
Fetching 15m data...
Calculating indicators...
Processing candles...
Saved image pair 8/5000
Saved image pair 9/5000
Saved image pair 10/5000
Saved image pair 11/5000
Fetching 30m data...
Calculating indicators...
Processing candles...
Saved image pair 12/5000
Saved image pair 13/5000
Fetching 1h data...
Calculating indicators...
Processing candles...
Saved image pair 14/5000
Saved image pair 15/5000
Saved image pair 16/5000
Saved image pair 17/5000
Saved image pair 18/5000
Saved image pair 19/5000
Saved image pair 20/5000
Saved image pair 21/5000
Saved image pair 22/5000
Saved image pair 23/5000
Saved image pair 24/5000
Saved image pair 25/5000
Saved image pair 26/5000
Saved image pair 27/5000
Saved image pair 28/5000
Saved

ERROR:yfinance:$^CNXMIDCAP: possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")


Not enough data: 0 candles
Fetching 15m data...


ERROR:yfinance:$^CNXMIDCAP: possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")


Not enough data: 0 candles
Fetching 30m data...


ERROR:yfinance:$^CNXMIDCAP: possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")


Not enough data: 0 candles
Fetching 1h data...


ERROR:yfinance:$^CNXMIDCAP: possibly delisted; no price data found  (period=730d) (Yahoo error = "No data found, symbol may be delisted")


Not enough data: 0 candles

Processing Gold Futures
Fetching 5m data...
Calculating indicators...
Processing candles...
Saved image pair 3478/5000
Saved image pair 3479/5000
Saved image pair 3480/5000
Fetching 15m data...
Calculating indicators...
Processing candles...
Saved image pair 3481/5000
Saved image pair 3482/5000
Fetching 30m data...
Calculating indicators...
Processing candles...
Saved image pair 3483/5000
Saved image pair 3484/5000
Fetching 1h data...
Calculating indicators...
Processing candles...
Saved image pair 3485/5000
Saved image pair 3486/5000
Saved image pair 3487/5000
Saved image pair 3488/5000
Saved image pair 3489/5000
Saved image pair 3490/5000
Saved image pair 3491/5000
Saved image pair 3492/5000
Saved image pair 3493/5000
Saved image pair 3494/5000
Saved image pair 3495/5000
Saved image pair 3496/5000
Saved image pair 3497/5000
Saved image pair 3498/5000
Saved image pair 3499/5000
Saved image pair 3500/5000
Saved image pair 3501/5000
Saved image pair 3502/5000

In [ ]:
class TradingDataCollector:
    def __init__(self):
        # Initialize instruments first
        self.instruments = {
                  # US Large Caps & Tech
                  'AAPL': 'Apple', 'MSFT': 'Microsoft', 'GOOGL': 'Google',
                  'AMZN': 'Amazon', 'NVDA': 'NVIDIA', 'META': 'Meta',
                  'TSLA': 'Tesla', 'AMD': 'AMD', 'JPM': 'JPMorgan',
                  'V': 'Visa', 'MA': 'Mastercard', 'PYPL': 'PayPal',
                  'NFLX': 'Netflix', 'DIS': 'Disney', 'ADBE': 'Adobe',
                  'ORCL': 'Oracle', 'CRM': 'Salesforce', 'INTC': 'Intel',
                  'BAC': 'Bank of America', 'WMT': 'Walmart',
                  'PG': 'P&G', 'KO': 'Coca-Cola', 'PEP': 'PepsiCo',
                  'MCD': 'McDonalds', 'NKE': 'Nike',

                  # Indian Large Caps (High Volume)
                  'RELIANCE.NS': 'Reliance Industries', 'TCS.NS': 'TCS', 'INFY.NS': 'Infosys',
                  'HDFCBANK.NS': 'HDFC Bank', 'ICICIBANK.NS': 'ICICI Bank',
                  'SBIN.NS': 'State Bank of India', 'BHARTIARTL.NS': 'Bharti Airtel',
                  'KOTAKBANK.NS': 'Kotak Mahindra Bank', 'LT.NS': 'Larsen & Toubro',
                  'ITC.NS': 'ITC', 'HINDUNILVR.NS': 'Hindustan Unilever', 'ASIANPAINT.NS': 'Asian Paints',
                  'MARUTI.NS': 'Maruti Suzuki', 'BAJFINANCE.NS': 'Bajaj Finance',
                  'ADANIENT.NS': 'Adani Enterprises', 'TITAN.NS': 'Titan', 'ULTRACEMCO.NS': 'UltraTech Cement',
                  'WIPRO.NS': 'Wipro', 'M&M.NS': 'Mahindra & Mahindra',

                  # Major Stock Indices (High Volume)
                  '^GSPC': 'S&P 500', '^DJI': 'Dow Jones Industrial Average', '^IXIC': 'Nasdaq Composite',
                  '^RUT': 'Russell 2000', '^VIX': 'CBOE Volatility Index', '^FTSE': 'FTSE 100',
                  '^GDAXI': 'DAX 40', '^FCHI': 'CAC 40', '^N225': 'Nikkei 225',
                  '^HSI': 'Hang Seng Index', '^AXJO': 'ASX 200', '^STI': 'Straits Times Index',
                  '^NSEI': 'Nifty 50', '^BSESN': 'Sensex', '^NSEBANK': 'Nifty Bank',
                  '^CNXIT': 'Nifty IT', '^CNXMIDCAP': 'Nifty Midcap 100',

                  # High-Volume Commodities & Metals
                  'GC=F': 'Gold Futures', 'SI=F': 'Silver Futures', 'HG=F': 'Copper Futures',
                  'PL=F': 'Platinum Futures', 'PA=F': 'Palladium Futures', 'CL=F': 'Crude Oil Futures',
                  'NG=F': 'Natural Gas Futures', 'ZC=F': 'Corn Futures', 'ZW=F': 'Wheat Futures',
                  'ZS=F': 'Soybean Futures', 'ZL=F': 'Soybean Oil Futures', 'KC=F': 'Coffee Futures',
                  'CT=F': 'Cotton Futures',

                  # Forex Major Pairs (High Liquidity)
                  'EURUSD=X': 'EUR/USD', 'GBPUSD=X': 'GBP/USD', 'USDJPY=X': 'USD/JPY',
                  'AUDUSD=X': 'AUD/USD', 'USDCAD=X': 'USD/CAD', 'NZDUSD=X': 'NZD/USD',
                  'USDCHF=X': 'USD/CHF', 'EURJPY=X': 'EUR/JPY', 'GBPJPY=X': 'GBP/JPY',
                  'EURGBP=X': 'EUR/GBP', 'USDHKD=X': 'USD/HKD', 'USDSGD=X': 'USD/SGD',
                  'USDZAR=X': 'USD/ZAR', 'USDNOK=X': 'USD/NOK', 'EURCAD=X': 'EUR/CAD',
                  'AUDJPY=X': 'AUD/JPY', 'USDSEK=X': 'USD/SEK', 'EURCHF=X': 'EUR/CHF',

                  # Crypto (High Volume)
                  'BTC-USD': 'Bitcoin', 'ETH-USD': 'Ethereum', 'BNB-USD': 'Binance',
                  'SOL-USD': 'Solana', 'XRP-USD': 'Ripple', 'ADA-USD': 'Cardano',
                  'DOGE-USD': 'Dogecoin', 'MATIC-USD': 'Polygon', 'LINK-USD': 'Chainlink',
                  'DOT-USD': 'Polkadot', 'AVAX-USD': 'Avalanche', 'SHIB-USD': 'Shiba Inu',
                  'UNI-USD': 'Uniswap', 'ATOM-USD': 'Cosmos', 'TRX-USD': 'Tron',
                  'LTC-USD': 'Litecoin', 'BCH-USD': 'Bitcoin Cash', 'XLM-USD': 'Stellar',
                  'ICP-USD': 'Internet Computer', 'APT-USD': 'Aptos', 'ARB-USD': 'Arbitrum',
                  'OP-USD': 'Optimism'

        }

        # Adjusted timeframes to comply with Yahoo Finance limits
        self.timeframes = {
            '5m': '60d',    # Maximum allowed for 5m
            '15m': '60d',   # Maximum allowed for 15m
            '30m': '60d',   # Maximum allowed for 30m
            '1h': '730d'    # Maximum allowed for 1h (2 years)
        }

        # Main directories - Changed from 'buyclass' to 'sellclass'
        self.base_dir = 'sellclass'
        self.dirs = {
            'input': f'{self.base_dir}/sellinput',
            'output': f'{self.base_dir}/selloutput'
        }

        # Create directories
        for path in self.dirs.values():
            os.makedirs(path, exist_ok=True)

        # Sequential image counter
        self.image_counter = 1

        # Date tracking
        self.collected_dates = {
            f"{symbol}_{timeframe}": set()
            for symbol in self.instruments
            for timeframe in self.timeframes
        }

    def calculate_indicators(self, df):
        """Calculate technical indicators for analysis"""
        try:
            # Make sure we have OHLCV data
            required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
            if not all(col in df.columns for col in required_columns):
                print("Missing required price data columns")
                return None

            # Calculate EMAs
            df['EMA9'] = df['Close'].ewm(span=9, adjust=False).mean()
            df['EMA21'] = df['Close'].ewm(span=21, adjust=False).mean()
            df['EMA55'] = df['Close'].ewm(span=55, adjust=False).mean()
            df['EMA89'] = df['Close'].ewm(span=89, adjust=False).mean()

            # Calculate Stochastic
            period = 14
            df['LowestLow'] = df['Low'].rolling(window=period).min()
            df['HighestHigh'] = df['High'].rolling(window=period).max()
            df['%K'] = ((df['Close'] - df['LowestLow']) /
                       (df['HighestHigh'] - df['LowestLow'])) * 100
            df['%D'] = df['%K'].rolling(window=3).mean()

            # Calculate ATR for volatility
            df['TR'] = np.maximum(
                df['High'] - df['Low'],
                np.maximum(
                    abs(df['High'] - df['Close'].shift()),
                    abs(df['Low'] - df['Close'].shift())
                )
            )
            df['ATR'] = df['TR'].rolling(window=14).mean()

            # Calculate Fibonacci levels
            df['High_Roll'] = df['High'].rolling(window=55).max()
            df['Low_Roll'] = df['Low'].rolling(window=55).min()
            range_size = df['High_Roll'] - df['Low_Roll']

            # Fibonacci retracement levels
            df['Fib_618'] = df['High_Roll'] - (range_size * 0.618)
            df['Fib_500'] = df['High_Roll'] - (range_size * 0.500)
            df['Fib_382'] = df['High_Roll'] - (range_size * 0.382)

            return df

        except Exception as e:
            print(f"Error calculating indicators: {str(e)}")
            return None

    def check_ema_condition(self, df, index):
        """Check if EMAs are properly aligned for sell (reversed from buy)"""
        try:
            current = df.iloc[index]
            # Reversed condition for sell signals (EMAs descending)
            return (current['EMA9'] < current['EMA21'] <
                   current['EMA55'] < current['EMA89'])
        except Exception as e:
            print(f"Error checking EMA condition: {str(e)}")
            return False

    def check_golden_zone(self, df, index):
        """Check if price is in the upper golden zone for sell signals"""
        try:
            current = df.iloc[index]
            price = current['Close']
            # Calculate upper golden zone (0.618-0.382 from the top)
            upper_fib_618 = current['Low_Roll'] + ((current['High_Roll'] - current['Low_Roll']) * 0.618)
            upper_fib_382 = current['Low_Roll'] + ((current['High_Roll'] - current['Low_Roll']) * 0.382)
            return (upper_fib_382 <= price <= upper_fib_618)
        except Exception as e:
            print(f"Error checking golden zone: {str(e)}")
            return False

    def check_discount_zone(self, df, index):
        """Check if price is in the upper discount zone for sell signals"""
        try:
            current = df.iloc[index]
            price = current['Close']
            # Calculate upper discount zone (0.786-0.618 from the top)
            upper_fib_786 = current['Low_Roll'] + ((current['High_Roll'] - current['Low_Roll']) * 0.786)
            upper_fib_618 = current['Low_Roll'] + ((current['High_Roll'] - current['Low_Roll']) * 0.618)
            return (upper_fib_618 <= price <= upper_fib_786)
        except Exception as e:
            print(f"Error checking discount zone: {str(e)}")
            return False

    def check_stochastic(self, df, index):
        """Check if stochastic indicates overbought conditions for sell signals"""
        try:
            current = df.iloc[index]
            # Changed from oversold (20-40) to overbought (60-80)
            return (60 <= current['%K'] <= 80 and
                   60 <= current['%D'] <= 80)
        except Exception as e:
            print(f"Error checking stochastic: {str(e)}")
            return False

    def check_ema_crossover(self, df, index):
        """Check for bearish EMA crossover (reversed from bullish)"""
        try:
            if index < 1:
                return False

            current = df.iloc[index]
            previous = df.iloc[index-1]

            # Reversed condition for bearish crossover
            return (previous['EMA9'] >= previous['EMA21'] and
                   current['EMA9'] < current['EMA21'])
        except Exception as e:
            print(f"Error checking EMA crossover: {str(e)}")
            return False

    def is_date_collected(self, symbol, timeframe, timestamp):
        """Check if we already collected data for this date"""
        key = f"{symbol}_{timeframe}"
        date_str = timestamp.strftime('%Y-%m-%d')

        if date_str in self.collected_dates[key]:
            return True

        self.collected_dates[key].add(date_str)
        return False

    def check_all_conditions(self, df, index):
        """Check all possible combinations of conditions"""
        try:
            # Base condition (Required)
            ema_ok = self.check_ema_condition(df, index)
            if not ema_ok:
                return False

            # Check all other conditions
            golden_zone = self.check_golden_zone(df, index)
            discount_zone = self.check_discount_zone(df, index)
            stoch_ok = self.check_stochastic(df, index)
            crossover = self.check_ema_crossover(df, index)

            # All possible combinations with base condition
            combinations = [
                (golden_zone,),                                    # 1+2
                (discount_zone,),                                 # 1+3
                (stoch_ok,),                                     # 1+4
                (crossover,),                                    # 1+5
                (golden_zone, discount_zone),                    # 1+2+3
                (golden_zone, stoch_ok),                        # 1+2+4
                (golden_zone, crossover),                       # 1+2+5
                (discount_zone, stoch_ok),                      # 1+3+4
                (discount_zone, crossover),                     # 1+3+5
                (stoch_ok, crossover),                         # 1+4+5
                (golden_zone, discount_zone, stoch_ok),        # 1+2+3+4
                (golden_zone, discount_zone, crossover),       # 1+2+3+5
                (golden_zone, stoch_ok, crossover),           # 1+2+4+5
                (discount_zone, stoch_ok, crossover),         # 1+3+4+5
                (golden_zone, discount_zone, stoch_ok, crossover) # 1+2+3+4+5
            ]

            # Return True if any combination is satisfied
            return any(all(cond for cond in combo) for combo in combinations)

        except Exception as e:
            print(f"Error checking conditions: {str(e)}")
            return False

    def save_chart_images(self, df, index):
        """Save input and output images"""
        try:
            # Input image (200 candles)
            start_idx = max(0, index - 199)
            input_data = df.iloc[start_idx:index+1]

            if len(input_data) != 200:
                return False

            # Output image (200 candles: 100 deleted + 100 future)
            if index + 100 >= len(df):
                return False

            output_data = df.iloc[index-99:index+101]
            if len(output_data) != 200:
                return False

            # Chart style
            mc = mpf.make_marketcolors(
                up='green',
                down='red',
                edge='inherit',
                wick='inherit'
            )
            s = mpf.make_mpf_style(
                marketcolors=mc,
                gridstyle='',
                y_on_right=True
            )

            # Save input image
            fig_input, _ = mpf.plot(
                input_data,
                type='candle',
                style=s,
                volume=False,
                returnfig=True,
                figsize=(10, 10),
                tight_layout=True,
                show_nontrading=False
            )

            input_path = os.path.join(self.dirs['input'], f'inputimg{self.image_counter}.png')
            fig_input.savefig(input_path, dpi=51.2, bbox_inches='tight', pad_inches=0.1)
            plt.close(fig_input)

            # Save output image
            fig_output, _ = mpf.plot(
                output_data,
                type='candle',
                style=s,
                volume=False,
                returnfig=True,
                figsize=(10, 10),
                tight_layout=True,
                show_nontrading=False
            )

            output_path = os.path.join(self.dirs['output'], f'outputimg{self.image_counter}.png')
            fig_output.savefig(output_path, dpi=51.2, bbox_inches='tight', pad_inches=0.1)
            plt.close(fig_output)

            # Resize both images
            for path in [input_path, output_path]:
                img = Image.open(path)
                img = img.resize((512, 512), Image.Resampling.LANCZOS)
                img.save(path)

            self.image_counter += 1
            return True
        except Exception as e:
            print(f"Error saving charts: {str(e)}")
            return False

    def collect_data(self):
        """Main data collection process"""
        print("Starting sell signal data collection process...")
        start_time = time.time()

        for symbol in self.instruments:
            if self.image_counter > 5000:
                break

            print(f"\nProcessing {self.instruments[symbol]}")

            for timeframe, period in self.timeframes.items():
                try:
                    print(f"Fetching {timeframe} data...")
                    ticker = yf.Ticker(symbol)
                    df = ticker.history(interval=timeframe, period=period)

                    if len(df) < 300:
                        print(f"Not enough data: {len(df)} candles")
                        continue

                    print("Calculating indicators...")
                    df = self.calculate_indicators(df)
                    if df is None:
                        continue

                    print("Processing candles...")
                    for i in range(200, len(df)-100):
                        if self.image_counter > 5000:
                            break

                        # Check date duplication
                        timestamp = pd.Timestamp(df.index[i])
                        if self.is_date_collected(symbol, timeframe, timestamp):
                            continue

                        # Check all condition combinations
                        if self.check_all_conditions(df, i):
                            # Save synchronized images
                            if self.save_chart_images(df, i):
                                print(f"Saved sell image pair {self.image_counter}/5000")

                except Exception as e:
                    print(f"Error processing {symbol} {timeframe}: {str(e)}")
                    continue

        end_time = time.time()
        print(f"\nCollection Complete!")
        print(f"Total sell image pairs collected: {self.image_counter-1}")
        print(f"Time taken: {(end_time - start_time) / 60:.2f} minutes")

if __name__ == "__main__":


    collector = TradingDataCollector()
    collector.collect_data()

Starting sell signal data collection process...

Processing Apple
Fetching 5m data...
Calculating indicators...
Processing candles...
Saved sell image pair 2/5000
Saved sell image pair 3/5000
Saved sell image pair 4/5000
Saved sell image pair 5/5000
Saved sell image pair 6/5000
Saved sell image pair 7/5000
Saved sell image pair 8/5000
Fetching 15m data...
Calculating indicators...
Processing candles...
Saved sell image pair 9/5000
Saved sell image pair 10/5000
Saved sell image pair 11/5000
Saved sell image pair 12/5000
Saved sell image pair 13/5000
Fetching 30m data...
Calculating indicators...
Processing candles...
Saved sell image pair 14/5000
Saved sell image pair 15/5000
Saved sell image pair 16/5000
Fetching 1h data...
Calculating indicators...
Processing candles...
Saved sell image pair 17/5000
Saved sell image pair 18/5000
Saved sell image pair 19/5000
Saved sell image pair 20/5000
Saved sell image pair 21/5000
Saved sell image pair 22/5000
Saved sell image pair 23/5000
Saved se

ERROR:yfinance:$^CNXMIDCAP: possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")


Not enough data: 0 candles
Fetching 15m data...


ERROR:yfinance:$^CNXMIDCAP: possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")


Not enough data: 0 candles
Fetching 30m data...


ERROR:yfinance:$^CNXMIDCAP: possibly delisted; no price data found  (period=60d) (Yahoo error = "No data found, symbol may be delisted")


Not enough data: 0 candles
Fetching 1h data...


ERROR:yfinance:$^CNXMIDCAP: possibly delisted; no price data found  (period=730d) (Yahoo error = "No data found, symbol may be delisted")


Not enough data: 0 candles

Processing Gold Futures
Fetching 5m data...
Calculating indicators...
Processing candles...
Saved sell image pair 2830/5000
Saved sell image pair 2831/5000
Saved sell image pair 2832/5000
Saved sell image pair 2833/5000
Saved sell image pair 2834/5000
Fetching 15m data...
Calculating indicators...
Processing candles...
Saved sell image pair 2835/5000
Saved sell image pair 2836/5000
Saved sell image pair 2837/5000
Fetching 30m data...
Calculating indicators...
Processing candles...
Saved sell image pair 2838/5000
Saved sell image pair 2839/5000
Fetching 1h data...
Calculating indicators...
Processing candles...
Saved sell image pair 2840/5000
Saved sell image pair 2841/5000
Saved sell image pair 2842/5000
Saved sell image pair 2843/5000
Saved sell image pair 2844/5000
Saved sell image pair 2845/5000
Saved sell image pair 2846/5000
Saved sell image pair 2847/5000
Saved sell image pair 2848/5000
Saved sell image pair 2849/5000
Saved sell image pair 2850/5000
Sa